# Decoding MPP encodings
In this notebook I am looking into the feasibility of 
decoding a MPP encoding. 

## Setup

In [ ]:
import pandas as pd
import numpy as np
import scipy
from scipy.sparse import csr_array
from sklearn.cluster import DBSCAN
import shapely
import shapely.wkt
import plotly
import plotly.subplots
from plotly.graph_objs import Scatter, Figure

from geo_encodings import MPPEncoder
from geo_encodings import draw_shape

## Create an embedding for a region

In [ ]:
x0, y0 = 0, 0
x1, y1 = 400, 300
resolution = 50
scale = 50
encoder = MPPEncoder(region=[x0, y0, x1, y1], resolution=resolution, scale=scale, center=True)
print('%d reference points in encoder' % len(encoder))

## Create geometries

In [ ]:
# geom = shapely.wkt.loads('POINT(287 122)')
# geom = shapely.wkt.loads('LINESTRING(40 70, 160 110, 213 190, 285 205)')
# geom = shapely.wkt.loads('POLYGON((70 20, 210 105, 127 240, 70 20))')
# geom = shapely.wkt.loads('MULTIPOINT(287 122, 113 207)')
geom = shapely.wkt.loads('MULTILINESTRING((40 70, 160 110, 213 190, 285 205), (220 140, 305 110, 345 30))')
# geom = shapely.wkt.loads('MULTIPOLYGON(((70 20, 210 105, 127 240, 70 20)), ((260 225, 270 170, 330 95, 335 185, 260 225)))')


In [ ]:
enc = encoder.encode(geom)
e = enc.values()
distances = -np.log(e) * encoder.scale

In [ ]:
def circle_xy(x0, y0, radius):
    tt = np.linspace(0, 2.0 * 3.14159)
    xx = x0 + radius * np.cos(tt)
    yy = y0 + radius * np.sin(tt)
    return xx, yy

In [ ]:
fig = plotly.subplots.make_subplots(1, 2)

# ==============
trace = Scatter(
    x=encoder.ref_x, y=encoder.ref_y, name='reference points', 
    mode='markers', marker={'color': 'gray', 'size': 5, 'symbol': 'cross'}
)
fig.append_trace(trace, 1, 1)
draw_shape(geom, fig, 1, 1, linewidth=2)
fig['layout']['xaxis1']['range'] = [0, 400]
fig['layout']['yaxis1']['range'] = [0, 300]


# ==============
# Draw the circular "exclusion zones" around each reference point.
for i in range(len(encoder)):
    circle_x, circle_y = circle_xy(encoder.ref_x[i], encoder.ref_y[i], distances[i])
    trace = Scatter(
        x=circle_x, y=circle_y, name='polygon', 
        mode='lines', marker={'color': 'red'}, 
        fill='toself', fillcolor = 'rgba(255, 0, 0, 0.1)',
        line={'width': 1}
    )
    fig.append_trace(trace, 1, 2)

trace = Scatter(
    x=encoder.ref_x, y=encoder.ref_y, name='reference points', 
    mode='markers', marker={'color': 'gray', 'size': 5, 'symbol': 'cross'}
)
fig.append_trace(trace, 1, 2)
draw_shape(geom, fig, 1, 2, linewidth=2)

fig['layout']['width'] = 950
fig['layout']['height'] = 400
fig['layout']['xaxis2']['range'] = [0, 400]
fig['layout']['yaxis2']['range'] = [0, 300]

fig